In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from skimage import io, transform
from skimage.transform import rotate as rotate_transform
from skimage.util import random_noise
from skimage.filters import gaussian
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
# Define the resnet50 model
class resnet50(nn.Module):
    def __init__(self, num_classes=22):
        super().__init__()
        self.model_name = 'resnet50'
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

# Instantiate the model
network = resnet50()
network.load_state_dict(torch.load('C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Dryad/model_resnet50_regressor_finetuned.pth', map_location=torch.device('cpu')))
network.eval()

C:\Users\nuhrr\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nuhrr\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


resnet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [4]:
# Load the Excel file
xlsx_dir = 'C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/Results/final_unclean.xlsx'
df = pd.read_excel(xlsx_dir)

# Define the V22 path
V22_path = "C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/V22"

In [5]:
# Define the unnormalize class
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor

In [6]:
# Define pages of interest
pages = [779, 780, 782, 868]

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    page = int(str(row['vpn'])[2:6])
    img_num = str(row['vpn'])[6:8]

    if page in pages:
        print(f"Processing page: {page}, image number: {img_num}")

        new_dir = f'C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/V22_plots/{page}'
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)

        left_img_path = os.path.join(V22_path, f'page{page}', 'Picture', f'V22P{page}L{img_num}L.bmp')
        right_img_path = os.path.join(V22_path, f'page{page}', 'Picture', f'V22P{page}L{img_num}R.bmp')

        if os.path.exists(left_img_path) and os.path.exists(right_img_path):
            # Process the left image
            im_L_ = Image.open(left_img_path)
            with torch.no_grad():
                im_L = TF.resize(im_L_, (224, 224))
                im_L = TF.to_tensor(im_L).unsqueeze(0)
                im_L = TF.normalize(im_L, [0.5], [0.5])
                predictionL = (network(im_L) + 0.5)
                predictionL = predictionL.view(-1, 11, 2) * torch.tensor([1280, 1024])
                predictionL = predictionL.squeeze().numpy()

                plt.figure(figsize=(10, 10))
                plt.imshow(im_L_)
                plt.scatter(predictionL[:, 0], predictionL[:, 1], s=15, color='g')
                plt.savefig(os.path.join(new_dir, f'V22P{page}L{img_num}L.png'))
                plt.close()

            # Process the right image
            im_R_ = Image.open(right_img_path).transpose(method=Image.FLIP_LEFT_RIGHT)
            with torch.no_grad():
                im_R = TF.resize(im_R_, (224, 224))
                im_R = TF.to_tensor(im_R).unsqueeze(0)
                im_R = TF.normalize(im_R, [0.5], [0.5])
                predictionR = (network(im_R) + 0.5)
                predictionR = predictionR.view(-1, 11, 2) * torch.tensor([1280, 1024])
                predictionR = predictionR.squeeze().numpy()

                plt.figure(figsize=(10, 10))
                plt.imshow(im_R_)
                plt.scatter(predictionR[:, 0], predictionR[:, 1], s=15, color='g')
                plt.savefig(os.path.join(new_dir, f'V22P{page}L{img_num}R.png'))
                plt.close()
        else:
            print(f"Images for page {page}, image number {img_num} not found.")


Processing page: 868, image number: 01
Processing page: 868, image number: 02
Processing page: 868, image number: 03
Processing page: 868, image number: 04
Processing page: 868, image number: 05
Processing page: 868, image number: 06
Processing page: 868, image number: 07
Processing page: 868, image number: 08
Processing page: 868, image number: 09
Processing page: 868, image number: 10
Processing page: 868, image number: 11
Processing page: 868, image number: 12
Processing page: 868, image number: 13
Processing page: 868, image number: 14
Processing page: 868, image number: 15
Processing page: 868, image number: 16
Processing page: 868, image number: 17
Processing page: 868, image number: 18
Processing page: 868, image number: 19
Processing page: 868, image number: 20
Images for page 868, image number 20 not found.
